# Análisis Exploratorio de Datos (EDA) - Online Retail

En este cuaderno, realizaremos un **análisis paso a paso** de un dataset de facturación que contiene columnas como:
 - **InvoiceNo**: Número de factura
 - **StockCode**: Código del producto
 - **Description**: Descripción del producto
 - **Quantity**: Cantidad vendida (puede ser negativa en devoluciones)
 - **InvoiceDate**: Fecha de la factura
 - **UnitPrice**: Precio unitario de cada artículo
 - **CustomerID**: ID del cliente (puede haber nulos si no se registró)
 - **Country**: País donde se realizó la compra

El objetivo es **entender** el comportamiento de las ventas, los países más relevantes, los productos más comprados, y otras métricas de interés.

In [ ]:
!wget https://github.com/javierherrera1996/lecture_analytics/raw/main/datasets/ecomerce_customers_segmentation.zip
!unzip ecomerce_customers_segmentation.zip

## 1. Importar librerías y cargar datos
Asegúrate de tener el archivo CSV en la misma carpeta, o ajusta la ruta de `pd.read_csv` según tu necesidad.

In [ ]:
import pandas as pd
import numpy as np

# Carga del dataset (ajusta el nombre a tu archivo)
df = pd.read_csv('data.csv',encoding="latin-1") # Puedes necesitar encoding='latin1' si hay caracteres especiales
pd.set_option('display.max_columns', None)
# Mostramos las primeras filas para inspección
df.head()

## 2. Información general del dataset
Aquí veremos:
- Dimensiones del DataFrame
- Tipos de datos
- Presencia de valores nulos
- Estadísticos descriptivos de las columnas numéricas

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
# Estadísticos descriptivos
df.isnull().sum()

**Observación**: Algunas facturas pueden ser devoluciones si `Quantity` es negativa. Además, puede haber filas sin `CustomerID`.

## 3. Limpieza básica y transformación
### 3.1. Revisar y eliminar posibles filas con datos problemáticos
 - Filas donde `Description` esté vacía
 - Filas con `Quantity == 0` (puede haber poco sentido en un registro así)
 - Filas con `UnitPrice <= 0` (podría ser error o promoción anómala)
 - Convertir la columna `InvoiceDate` a formato fecha, para luego analizar tiempos

In [ ]:
df=df.dropna(subset=["Description"])

In [ ]:
df=df[df["Quantity"]!=0]

In [ ]:
df=df[df["UnitPrice"]>0]

In [ ]:
df["InvoiceDate"]=pd.to_datetime(df["InvoiceDate"],format="DD/MM/YYYY HH:MM")

In [ ]:
df["InvoiceDate"][0]

## 4. Creación de columna "TotalPrice"
Para conocer el valor total de cada línea de venta, calculamos `Quantity * UnitPrice`.

In [ ]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

## 5. Análisis paso a paso
A continuación, veremos distintos análisis que **paso a paso** nos ayudarán a entender mejor el negocio.

### 5.1. ¿Cuántas transacciones totales hay y cuántos clientes diferentes?
Esto da un primer vistazo al tamaño del negocio en términos de facturas y base de clientes.

In [ ]:
transacciones_totales = df['InvoiceNo'].count()
print(f"Número total de transacciones: {transacciones_totales}")
clientes_unicos = df['CustomerID'].nunique()
print(f"Número de clientes únicos: {clientes_unicos}")

### 5.2. ¿Cuáles son los países más frecuentes en las ventas?
Podemos contar cuántas líneas de factura vienen de cada país. Esto ayuda a entender nuestro alcance geográfico.

In [ ]:
df.groupby("Country").count()["InvoiceNo"]

> Tip: Si queremos ver el total en **monetario** por país, podemos agrupar por `Country` y sumar `TotalPrice`.

In [ ]:
df.groupby("Country").sum()["TotalPrice"]

### 5.3. ¿Cuáles son los productos más vendidos?
Aquí vamos a usar `Description` para ver cuántas líneas se han vendido (o la suma total de `Quantity`).

In [ ]:
productos_por_lineas = df['Description'].value_counts()
print("Productos más vendidos (por número de líneas de factura):")
print(productos_por_lineas.head())

productos_por_cantidad = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False)
print("\nProductos más vendidos (por cantidad total):")
print(productos_por_cantidad.head())

### 5.4. ¿Cuál es el total de ingresos (revenue) y el ticket promedio?
 - El ingreso total es la suma de **TotalPrice**.
 - El ticket promedio podemos calcularlo como el promedio de **TotalPrice** dentro de cada factura (o línea).

In [ ]:
ingreso_total = df['TotalPrice'].sum()
print(f"Ingreso total: {ingreso_total:.2f}")

ticket_promedio_linea = df['TotalPrice'].mean()
print(f"Ticket promedio por línea de factura: {ticket_promedio_linea:.2f}")

> Para un ticket promedio a nivel de **factura**, debemos agrupar por `InvoiceNo`, sumar `TotalPrice` y luego promediar.

In [ ]:
ticket_promedio_factura = df.groupby('InvoiceNo')['TotalPrice'].sum().mean()
print(f"Ticket promedio por factura: {ticket_promedio_factura:.2f}")

### 5.6. ¿Tenemos devoluciones?
Como `Quantity` puede ser negativa, **representa** devoluciones. Veamos cuántas hay y el impacto en valor.

In [ ]:
devoluciones_df = df[df['Quantity'] < 0]

numero_devoluciones = devoluciones_df.shape[0]
print(f"Número total de líneas de factura con devoluciones: {numero_devoluciones}")

valor_devoluciones = devoluciones_df['TotalPrice'].sum()
print(f"Impacto total en valor de las devoluciones: {valor_devoluciones:.2f}")

### 5.7. ¿Cuántos clientes realizan compras repetidas?
Podemos ver cuántos clientes compran más de una vez.

In [ ]:
facturas_por_cliente = df.groupby('CustomerID')['InvoiceNo'].nunique()

clientes_repetidos = facturas_por_cliente[facturas_por_cliente > 1]

numero_clientes_repetidos = clientes_repetidos.shape[0]

print(f"Número de clientes que realizan compras repetidas: {numero_clientes_repetidos}")

## 6. Posibles siguientes pasos
- **Crear segmentaciones** (p. ej. RFM: Recency, Frequency, Monetary) para clasificar clientes.
- **Analizar descuentos/promos** si tuvieras columnas extra.
- **Fusionar** con datos de marketing para calcular CAC (Customer Acquisition Cost) y LTV (Lifetime Value), etc.
- **Explorar** variables estacionales o días de la semana con mayor venta.

Con este **análisis paso a paso**, se obtiene un primer panorama de:
 - Cantidad de facturas y clientes.
 - Productos más vendidos.
 - Ingresos por país.
 - Montos totales y tickets promedio.
 - Distribución mensual de ventas.
 - Devoluciones y recurrencia de clientes.

¡A medida que se adquieran más habilidades en Python y análisis de datos, se pueden añadir gráficos, segmentaciones avanzadas y modelos predictivos!